In [2]:
ls

Spanish Tranlation Test.pdf* test_table.csv*
XYZ Spanish.ipynb            user_table.csv*
_DS_Store*


In [3]:
import pandas as pd
import numpy as np

from __future__ import division

In [4]:
dft = pd.read_csv("test_table.csv", parse_dates=True)

In [5]:
dfu = pd.read_csv("user_table.csv", parse_dates=True)

In [6]:
dft.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [7]:
dfu.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [8]:
dft.describe()

,user_id,conversion,test
count,453321.000000,453321.000000,453321.000000
mean,499937.514728,0.049579,0.476446
std,288665.193436,0.217073,0.499445
min,1.000000,0.000000,0.000000
25%,249816.000000,0.000000,0.000000
50%,500019.000000,0.000000,0.000000
75%,749522.000000,0.000000,1.000000
max,1000000.000000,1.000000,1.000000


In [9]:
# merge in customer data, will need country
dfj = pd.merge(dft, dfu)

In [50]:
# group A: control (Spaniard)
# group B: local writers versions
A = dfj[dfj.test == 0]
B = dfj[dfj.test == 1]

In [51]:
# conversion rate across all control records
A[A.conversion==1].size/A.size

0.055155571864205187

In [52]:
# conversion rate across all localized-translation records
B[B.conversion==1].size/B.size

0.043411161678422794

In [37]:
latam = dfj[dfj.country != 'Spain']   # latin america records only
spain = dfj[dfj.country == 'Spain']   # spain records only
latamA = latam[latam.test == 0]
latamB = latam[latam.test == 1]
spainA = spain[spain.test == 0]
spainB = spain[spain.test == 1]  # should be empty

"A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than any other Spanish-speaking country." This appears true in aggregate. Would be worth analyzing country by country.

In [57]:
# conversion rate for Spaniard words in Spain
spainA[spainA.conversion==1].size/spainA.size

0.079718821212004173

In [60]:
# conversion rate for non-spain group A (Spaniard)
latamA[latamA.conversion==1].size/latamA.size

0.048291790557495239

In [55]:
# conversion rate for non-spain group B (Local)
latamB[latamB.conversion==1].size/latamB.size

0.043411161678422794

"It appears that the non-localized translation was doing better!" It does appear that way. 

In [61]:
LSconv0 = latamA[latamA.conversion == 0]
LSconv1 = latamA[latamA.conversion == 1]
Lconv0 = latamB[latamB.conversion == 0]
Lconv1 = latamB[latamB.conversion == 1]

In [62]:
m = [[cno.size, cconv.size],[locno.size, locconv.size]]
m

[[2116344, 107388], [2476884, 112404]]

In [14]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(m)
print 'chisq = {}, p = {}, dof={}'.format(chi2, p, dof)

chisq = 653.774191708, p = 3.37522301248e-144, dof=1


In [15]:
dfg = dfj.groupby(["country", "test", "conversion"])

In [16]:
zz = np.array([[43,9],[44,4]])
chi2, p, dof, expected = chi2_contingency(zz)
print 'chisq = {}, p = {}, dof={}'.format(chi2, p, dof)

chisq = 1.0724852071, p = 0.300384770391, dof=1


In [17]:
la0.size/la1.size

NameError: name 'la0' is not defined

In [ ]:
s0.size/ s1.size

In [ ]:
dfg.size()

Null hypothesis: nothing interesting is happening. Local wording and Spain wording perform the same. 

Significance level: 95%

Run t-test to calculate p-value.

In [ ]:
dfg.country != 'Spain'

In [ ]:
dfg.country.type

A data scientist must question all aspects of the case presented, taking nothing for granted. We can begin with qualitative analysis, but must ultimately attempt to ground any theories through experimental testing, backed by rigorous quantitative analysis of what "the data suggest." 

"Spain based users have a much higher conversion rate". This is in fact seen in the data with some confidence. Note however that there can be be many aspects of the real world that account for this. Non-Spain users live in different countries, with different cultures, different economies. In some of those countries its winter when its summer in Spain.  We could hypothesize and gather additional data, like ping times, but working with the presented case, we can at least consider gender, country, etc. We should model the conversion data to determine the features that drive conversion, then see if those are different in Spain vs. Latin America. 

The experiment encompassed only 5 days and not an entire week. The latin american countries are in very different time zones. Its possible the data for Spain include 12 hours of weekend. 

"It appears the non-localized translation was doing better". 